In [1]:
import os
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np

mp_pose = mp.solutions.pose

# Initialize pose detector
pose = mp_pose.Pose()

def extract_landmarks(image_path):
    # Load image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Detect pose landmarks
    results = pose.process(image_rgb)
    
    # Extract landmarks
    if results.pose_landmarks:
        landmarks = [
            [lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark
        ]
        return np.array(landmarks).flatten()
    return None

# Loop through dataset and extract landmarks
data = []
labels = []
dataset_dir = "D:\yoga dataset\DATASET\TRAIN"
for label in os.listdir(dataset_dir):
    label_dir = os.path.join(dataset_dir, label)
    for image_file in os.listdir(label_dir):
        image_path = os.path.join(label_dir, image_file)
        landmarks = extract_landmarks(image_path)
        if landmarks is not None:
            data.append(landmarks)
            labels.append(label)

# Save to CSV
df = pd.DataFrame(data)
df['label'] = labels
df.to_csv('yoga_keypoints.csv', index=False)


In [2]:
pip install mediapipe


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load extracted data
data = pd.read_csv('yoga_keypoints.csv')
X = data.iloc[:, :-1].values
y = data['label'].values

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build a neural network
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(np.unique(y)), activation='softmax')  # Output layer
])

# Compile and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

# Save model
model.save('yoga_pose_classifier.h5')


Epoch 1/30
25/25 [==============================] - 1s 11ms/step - loss: 1.5749 - accuracy: 0.2853 - val_loss: 1.4121 - val_accuracy: 0.4922
Epoch 2/30
25/25 [==============================] - 0s 4ms/step - loss: 1.3868 - accuracy: 0.4060 - val_loss: 1.2054 - val_accuracy: 0.5337
Epoch 3/30
25/25 [==============================] - 0s 4ms/step - loss: 1.2057 - accuracy: 0.4838 - val_loss: 1.0660 - val_accuracy: 0.5803
Epoch 4/30
25/25 [==============================] - 0s 4ms/step - loss: 1.1162 - accuracy: 0.5370 - val_loss: 1.0371 - val_accuracy: 0.5803
Epoch 5/30
25/25 [==============================] - 0s 4ms/step - loss: 1.0193 - accuracy: 0.5772 - val_loss: 0.9347 - val_accuracy: 0.6114
Epoch 6/30
25/25 [==============================] - 0s 4ms/step - loss: 0.9845 - accuracy: 0.5901 - val_loss: 0.9212 - val_accuracy: 0.6736
Epoch 7/30
25/25 [==============================] - 0s 4ms/step - loss: 0.9622 - accuracy: 0.6070 - val_loss: 0.8789 - val_accuracy: 0.6632
Epoch 8/30
25/25 [=

C:\Users\User\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
pip install deap


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
from deap import base, creator, tools, algorithms
import random

# Define the fitness function
def fitness(individual):
    selected_features = [i for i, gene in enumerate(individual) if gene == 1]
    
    # If no features are selected, assign a low fitness score
    if len(selected_features) == 0:
        return 0,
    
    # Extract only the selected features
    X_selected = X_train[:, selected_features]

    # Dynamically create a model with input size matching the selected features
    dynamic_model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(len(selected_features),)),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')  # Adjust output layer based on your problem
    ])
    
    dynamic_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train the model on the selected features
    dynamic_model.fit(X_selected, y_train, epochs=5, batch_size=32, verbose=0)
    
    # Evaluate on the test set
    accuracy = dynamic_model.evaluate(X_test[:, selected_features], y_test, verbose=0)[1]
    return accuracy,



In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load dataset
data = pd.read_csv('yoga_keypoints.csv')
X = data.iloc[:, :-1].values  # Features (landmarks)
y = data['label'].values      # Labels

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8186528497409327


In [7]:
import matplotlib.pyplot as plt

import mediapipe as mp

# Define pose labels (must match the labels used during training)
pose_labels = ['DownwardDog', 'Warrior', 'Plank']

# Function to plot prediction probabilities
def plot_probabilities(probabilities):
    """
    Plot bar chart of pose prediction probabilities.
    """
    plt.figure(figsize=(8, 5))
    plt.bar(pose_labels, probabilities, color='skyblue')
    plt.xlabel('Yoga Poses', fontsize=12)
    plt.ylabel('Probability', fontsize=12)
    plt.title('Pose Prediction Probabilities', fontsize=14)
    plt.ylim(0, 1)
    plt.show()

# Function to plot pose landmarks
def plot_landmarks(image, landmarks):
    """
    Plot pose landmarks on the given image using Matplotlib.
    """
    mp_pose = mp.solutions.pose

    plt.figure(figsize=(8, 6))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    # Plot each landmark
    for lm in landmarks.landmark:
        plt.scatter(lm.x * image.shape[1], lm.y * image.shape[0], c='red', s=10)
    
    # Connect joints for visual clarity
    for connection in mp_pose.POSE_CONNECTIONS:
        start_idx, end_idx = connection
        x1 = landmarks.landmark[start_idx].x * image.shape[1]
        y1 = landmarks.landmark[start_idx].y * image.shape[0]
        x2 = landmarks.landmark[end_idx].x * image.shape[1]
        y2 = landmarks.landmark[end_idx].y * image.shape[0]
        plt.plot([x1, x2], [y1, y2], 'green', linewidth=2)
    
    plt.axis('off')
    plt.title('Pose Landmarks', fontsize=14)
    plt.show()

In [ ]:
from tensorflow.keras.models import load_model
import mediapipe as mp
import cv2
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Load the trained model
model = load_model('yoga_pose_classifier.h5')

# Define feedback rules for poses
feedback_rules = {
    "DownwardDog": "Keep your hips aligned with your shoulders.",
    "Warrior": "Ensure your front knee is directly above your ankle.",
    "Plank": "Keep your back straight."
}

def extract_landmarks(image):
    """
    Extracts pose landmarks from a given image matrix.
    """
    # Convert the frame to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    
    # Extract landmarks
    if results.pose_landmarks:
        landmarks = [
            [lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark
        ]
        return np.array(landmarks).flatten()
    return None

# Real-time detection and feedback
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Extract landmarks from the webcam frame
    landmarks = extract_landmarks(frame)
    if landmarks is not None:
        # Predict pose
        prediction = model.predict(np.expand_dims(landmarks, axis=0))
        pose_label = label_encoder.inverse_transform([np.argmax(prediction)])[0]
        
        # Display feedback
        feedback = feedback_rules.get(pose_label, "Good alignment!")
        cv2.putText(frame, f"Pose: {pose_label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, feedback, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    cv2.imshow('Yoga Pose Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 20ms/step


...

1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - ETA: 0s